In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import time 

In [2]:
dataset = pd.read_csv('new_appdata10.csv')

In [3]:
response = dataset['enrolled']

In [4]:
dataset = dataset.drop(columns = 'enrolled')

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(dataset, response, test_size = 0.2, random_state = 0 )

In [7]:
# train_identity = x_train['user']
# x_train = x_train.drop(columns = ['user'])
# test_identity = x_test['user']
# x_test = x_test.drop(columns = ['user'])
train_identity = x_train['user']
x_train = x_train.drop(columns = ['user'])
test_identity = x_test['user']
x_test = x_test.drop(columns = ['user'])

In [ ]:
x_train.columns


In [8]:
# from sklearn.preprocessing import StandardScaler
# sc_x = StandardScaler() # returns a numpy array with multiple dimensions
# x_train2 = pd.DataFrame(sc_x.fit_transform(x_train))
# x_test2 = pd.DataFrame(sc_x.transform(x_test))
# x_train2.columns = x_train.columns.values
# x_test2.columns = x_test.columns.values
# x_train2.index = x_train.index.values
# x_test2.index = x_test.index.values
# x_train = x_train2
# x_test = x_test2
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train2 = pd.DataFrame(sc_x.fit_transform(x_train))
x_test2 = pd.DataFrame(sc_x.transform(x_test))
x_train2.columns = x_train.columns.values
x_test2.columns = x_test.columns.values
x_train2.index = x_train.index.values
x_test2.index = x_test.index.values
x_train = x_train2
x_test = x_test2

ValueError: could not convert string to float: ' 20:00:00'

In [10]:
# Fitting Model to the Training Set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, penalty = 'l2')
classifier.fit(x_train, y_train)

ValueError: could not convert string to float: ' 18:00:00'

In [11]:
# Predicting Test Set
y_pred = classifier.predict(x_test)


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [12]:
# Evaluating Results
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)
precision_score(y_test, y_pred) # tp / (tp + fp)
recall_score(y_test, y_pred) # tp / (tp + fn)
f1_score(y_test, y_pred)

NameError: name 'y_pred' is not defined

In [13]:
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)
sns.heatmap(df_cm, annot=True, fmt='g')
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))


NameError: name 'cm' is not defined

In [14]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, x = x_train, y = y_train, cv = 10)
print("SVM Accuracy: %0.3f (+/- %0.3f)" % (accuracies.mean(), accuracies.std() * 2))

TypeError: cross_val_score() got an unexpected keyword argument 'x'

In [15]:
# Analyzing Coefficients
pd.concat([pd.DataFrame(dataset.drop(columns = 'user').columns, columns = ["features"]),
           pd.DataFrame(np.transpose(classifier.coef_), columns = ["coef"])
           ],axis = 1)

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [16]:
## Grid Search (Round 1)
from sklearn.model_selection import GridSearchCV

# Select Regularization Method
penalty = ['l1', 'l2']


In [17]:
# Create regularization hyperparameter space
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]


In [18]:
# Combine Parameters
parameters = dict(C=C, penalty=penalty)

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = "accuracy",
                           cv = 10,
                           n_jobs = -1)

In [20]:
t0 = time.time()
grid_search = grid_search.fit(x_train, y_train)
t1 = time.time()
print("Took %0.2f seconds" % (t1 - t0))

rf_best_accuracy = grid_search.best_score_
rf_best_parameters = grid_search.best_params_
rf_best_accuracy, rf_best_parameters



ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

In [25]:
# Select Regularization Method
penalty = ['l2', 'l3']


In [22]:
# Create regularization hyperparameter space
C = [0.1, 0.5, 0.9, 1, 2, 5]


In [26]:
# Combine Parameters
parameters = dict(C=C, penalty=penalty)

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = "accuracy",
                           cv = 10,
                           n_jobs = -1)
t0 = time.time()
grid_search = grid_search.fit(x_train, y_train)
t1 = time.time()
print("Took %0.2f seconds" % (t1 - t0))

rf_best_accuracy = grid_search.best_score_
rf_best_parameters = grid_search.best_params_
rf_best_accuracy, rf_best_parameters
grid_search.best_score_



ValueError: could not convert string to float: ' 18:00:00'

In [27]:
# Formatting Final Results
final_results = pd.concat([y_test, test_identity], axis = 1).dropna()
final_results['predicted_reach'] = y_pred
final_results = final_results[['user', 'enrolled', 'predicted_reach']].reset_index(drop=True)


NameError: name 'y_pred' is not defined